# 使用 OpenACC 进行 GPU 编程

此实验面向 Fortran 程序员。如果您更喜欢使用 C/C++，请单击 [此链接](../C/README.ipynb)

不要忘记查看其他 [OpenACC 资源](https://www.openacc.org/resources) 并加入我们的 [OpenACC Slack 频道](https://www.openacc.org/community#slack)，以分享您的经验并从社区获得更多帮助。

---

让我们执行下面的单元格以显示有关服务器上运行的 GPU 的信息。为此，请执行下面的单元格块，方法是将焦点放在它上面（用鼠标单击它），然后按 Ctrl-Enter，或按上面工具栏中的播放按钮。如果一切顺利，您应该会看到灰色单元格下方返回一些输出。

In [ ]:
!nvidia-smi

---

## 简介

我们这个实验的目标是学习如何在 GPU（图形处理单元）上运行我们的代码。

![development_cycle.png](../images/development_cycle.png)

这是 OpenACC 3 步开发周期。

**分析**您的代码，并预测哪里可以发现潜在的并行性。使用分析器帮助了解代码中正在发生的事情以及可能存在并行性的地方。

**并行化**您的代码，从最耗时的部分开始。专注于维护程序的正确结果。

**优化**您的代码，专注于最大化性能。在早期并行化期间，性能可能不会一次性提高。

我们目前正在处理**分析**步骤。我们将使用 NVIDIA Nsight Systems 分析器来了解一个相对简单的示例代码，然后再进行接下来的两个步骤。

---

## 运行代码（多核）

我们已经完成了实验室代码的基本多核实现。如果您希望使用 *parallel 指令*，请运行以下脚本。

In [ ]:
!cp ./solutions/multicore/laplace2d.f90 ./laplace2d.f90

---

如果您希望使用 kernels 指令，请运行以下脚本。

In [ ]:
!cp ./solutions/multicore/kernels/laplace2d.f90 ./laplace2d.f90

---

然后您可以通过运行以下脚本来运行多核代码。将创建一个名为**l​​aplace_multicore**的可执行文件。

In [ ]:
!nvfortran -fast -ta=multicore -Minfo=accel -o laplace_multicore laplace2d.f90 jacobi.f90 && ./laplace_multicore

### 可选：查看代码

如果您想复习一下我们正在处理的代码文件，可以使用下面的两个链接查看它们。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

### 可选：分析代码

如果您想使用 Nsight Systems 分析您的代码，请按照 **[Lab2](../../../module2/assets/Fortran/README.ipynb)** 中的说明进行操作，并将 NVTX 添加到您的代码中以检测应用程序。

---

## 可选：GPU（图形处理单元）简介

GPU 最初用于为视频游戏渲染计算机图形。虽然它们继续主导视频游戏硬件市场，但 GPU 也已被用作**高吞吐量并行硬件**。它们擅长同时做很多事情。

![cpu_with_gpu.png](../images/cpu_with_gpu.png)

与多核 CPU 类似，GPU 具有多个计算核心 - 与 CPU 核心相比，这些核心的功能较弱，因此单个核心的性能相对较差，尤其是在串行代码上。但是，典型的 GPU 有 1000 个这样的核心，当它们能够并行地一起解决问题时，我们可以看到一系列算法的速度比 CPU 快几个数量级。我们在加速应用程序中采用的编程模型是将计算成本高、可并行的代码部分卸载到 GPU 上，而代码的顺序部分将继续在 CPU 上运行。

GPU 就是所谓的 SIMD 架构（SIMD 代表：单指令、多数据）。这意味着 GPU 擅长执行单个计算机指令（例如数学指令或内存读/写）并将该指令应用于大量数据。最终，这意味着 GPU 可以同时执行数千个操作。此功能在某些方面类似于多核 CPU 架构，但当然，使用 GPU，我们可以拥有更多内核，并且指令会同时发送给在这些内核上运行的线程组，而不是每个线程。同样值得注意的是，GPU 内存的运行带宽通常比 CPU 内存高得多。许多应用程序都受带宽限制，即受从内存访问数据的速度限制，因此 GPU 也非常适合帮助加速这些应用程序。

![cpu_and_gpu_diagram.png](../images/cpu_and_gpu_diagram.png)

此图表示包含 CPU 和 GPU 的机器。我们可以看到，CPU 和 GPU 是两个完全独立的设备，通过 I/O 总线连接。该总线传统上是 PCI-e 总线，但 NVLink 是一种更新、更快的替代方案。这两个设备还具有独立的内存。这意味着在执行我们的程序期间，CPU 和 GPU 之间会传输一定量的数据。

---

## 使用 OpenACC 进行数据管理

在为 GPU 或类似架构编程时，设备内存与主机 CPU 内存不同，我们需要考虑主机和设备之间的数据管理。即使使用 NVLink，在 CPU 和 GPU 之间移动数据仍然需要时间成本，这可能会限制我们的应用程序性能，因此我们需要考虑缓解这种情况的方法，其中一些方法将在本实验和下一个实验中涉及。使用 OpenACC，程序员可以使用 OpenACC 数据指令和数据子句明确定义数据管理，或者，他们可以允许编译器为他们处理数据管理。

### 使用 OpenACC 数据子句

数据子句允许程序员指定主机和设备（或在我们的例子中是 CPU 和 GPU）之间的数据传输。让我们看一个不使用数据子句的示例。

```fortran
integer, dimension(:), allocatable :: A
allocate( A(N) )

!$acc parallel loop
do i=1,N
   A(i) = 0
enddo
```

我们在并行区域之外分配了一个数组 `A` 。这意味着 `A` 分配在 CPU 内存中。但是，我们在循环内部访问 `A` ，并且该循环包含在“并行”区域内。在该并行区域内， `A(i)` 试图访问 GPU 内存中的内存位置。我们没有在 GPU 上明确分配 `A` ，因此会发生以下两种情况之一。

1. 编译器将理解我们试图做什么，并自动将**A**从 CPU 复制到 GPU。

2. 程序将检查 GPU 内存中的数组**A**，如果找不到，则会抛出错误。

我们不必指望编译器能够解决这个问题，而是可以使用**数据子句**。

```fortran
integer, dimension(:), allocatable :: A
allocate( A(N) )

!$acc parallel loop copy(A)
do i=1,N
   A(i) = 0
enddo
```

我们将首先学习 `copy` 数据子句，因为它最容易使用。我们稍后会更详细地介绍语法，但现在，请理解，在包含 `copy` 数据子句后，我们的程序现在会将 `A` 的内容从 CPU 内存复制到 GPU 内存中。然后，在循环执行期间，它将正确地从 GPU 内存访问 `A`。并行区域完成后，我们的程序会将 `A` 从 GPU 内存复制回 CPU 内存。让我们再看一个直接的例子。

```fortran
integer, dimension(:), allocatable :: A
allocate( A(N) )

do i=1,N
    A(i) = 0;
enddo

!$acc parallel loop copy(A)
do i=1,N
    A(i) = 1;
enddo
```

现在我们有两个循环；第一个循环将在 CPU 上执行（因为它没有 OpenACC *并行指令*），第二个循环将在 GPU 上执行。数组“A”将在 CPU 上分配，然后执行第一个循环。此循环将“A”的内容全部设置为 0。然后遇到第二个循环；程序将数组“A”（全是 0）复制到 GPU 内存中。然后，我们将在 GPU 上执行第二个循环。这将编辑 GPU 的“A”副本，使其全是 1。

此时，我们有两个单独的“A”副本。CPU 副本全是 0，GPU 副本全是 1。现在，在并行区域完成后，程序将“A”从 GPU 复制回 CPU。在此复制之后，CPU 和 GPU 都将包含一个包含所有 1 的“A”副本。然后将释放“A”的 GPU 副本。

此图像提供了使用复制子句的另一个分步示例。

![copy_step_by_step](../images/copy_step_by_step.png)

我们还可以使用以下语法一次复制多个数组：

```fortran
!$acc parallel loop copy(A, B)
do i=1,N
   A(i) = B(i)
enddo
```

### 数组整形

数组的形状指定需要传输多少数据。让我们看一个例子：

```fortran
!$acc parallel loop copy(A(1:N))
do i=1,N
A(i) = 0
enddo
```

特别关注 `copy(A(1:N))` ，数组的形状在括号内定义。数组形状的语法是 **(starting_index:ending_index)** 。这意味着（在代码示例中）我们从数组 `A` 复制数据，从索引 1（数组的开头）开始，一直到数组的末尾 N。

我们也可以只复制数组的一部分：

```fortran
!$acc parallel copy(A(2:N-1))
```

这将复制 `A` 中除第一个和最后一个元素之外的所有元素。

最后，如果您未指定起始索引，则假定为 1。这意味着

```fortran
!$acc parallel loop copy(A(1:N))
```

等同于

```fortran
!$acc parallel loop copy(A(:N))
```

或

```fortran
!$acc parallel loop copy(A)
```

### 在 laplace 代码中包含数据子句

通过选择以下链接将**复制**数据子句添加到我们的 laplace 代码中：

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

然后，当您准备好时，您可以通过运行以下脚本来运行代码。这可能还不是直观明显的，但我们预计代码的性能会非常差。为此，我们在**明显较小的输入大小**上运行我们的 GPU 代码。如果您要在完整尺寸的输入上运行 GPU 代码，则需要几分钟才能运行。（**提示**：通过修改 [jacobi.f90](jacobi.f90) 中的 n 和 m 参数将网格的大小减小到 1024 x 1024。）

In [ ]:
!nvfortran -fast -ta=tesla -Minfo=accel -o laplace_data_clauses laplace2d.f90 jacobi.f90  && ./laplace_data_clauses 

如果您不确定自己的答案，可以点击 [此处](solutions/basic_data/laplace2d.f90) 查看解决方案。

### 可选：编译 GPU 代码

让我们通过运行 `nvaccelinfo` 命令来执行下面的单元格，以显示有关服务器上运行的 GPU 的信息，该命令随我们将使用的 NVIDIA HPC 编译器一起提供。

In [ ]:
!nvaccelinfo

这里包含了很多信息，但是我们只关注两点。

**托管内存：** 将告诉我们我们的 GPU 是否支持 CUDA 托管内存。我们将在稍后的实验中介绍托管内存。

**编译器选项：** 告诉我们要为哪个目标编译。之前，我们对多核代码使用了 `-ta=multicore` 标志。现在，为了针对我们的特定 GPU 进行编译，我们将用 `-ta=tesla` 替换它。

---

### 分析 GPU 代码

为了了解我们的程序为何表现如此糟糕，我们应该咨询我们的分析器。如前所述，如果我们使用默认的 4096x4096 阵列运行程序，该程序将需要几分钟才能运行。我建议您减小大小。尝试使用“1024 1024”作为参数。

In [ ]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_data_clauses ./laplace_data_clauses 

让我们查看一下剖析器的报告。剖析运行完成后，按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](laplace_data_clauses.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看剖析器报告，请参阅 [如何查看报告](../../../module2/assets/Fortran/README.ipynb) 部分。

这是您通过 GUI 打开剖析器报告后应该看到的视图。

![data_clause1.PNG](../images/data_clause1.png)

我们可以看到我们的“时间线”有很多事情要做。此时请随意浏览剖析器。放大会有所帮助，这样您就可以更好地查看信息。

![data_clause2.PNG](../images/data_clause2.png)

放大后，我们对程序内部发生的事情有了更好的了解。放大 while 循环的一次迭代，查看 `calcNext` 和 `swap` 的调用位置。您还可以看到它们之间有很多空间。现在可能很明显为什么我们的程序性能如此糟糕。我们的程序传输数据的时间（如 MemCpy 时间线所示）远远大于运行计算函数 `calcNext` 和 `swap` 所需的时间。为了提高我们的性能，我们需要尽量减少这些数据传输。

---

## 托管内存

![managed_memory.png](../images/cuda-unified-memory.svg)

与 OpenACC 中的许多东西一样，我们可以选择让编译器处理内存管理。我们可以通过自己管理内存来实现更好的性能，但是，允许编译器使用托管内存非常简单，并且将比我们之前的简单解决方案实现更好的性能。我们不需要对代码进行任何更改即可使托管内存正常工作。只需运行以下脚本即可。请记住，与之前不同，我们现在使用全尺寸 4096x4096 数组运行代码。

In [ ]:
!nvfortran -fast -ta=tesla:managed -Minfo=accel -o laplace_managed laplace2d.f90 jacobi.f90  && ./laplace_managed

### 可选：使用托管内存标志进行编译

只要 GPU 支持托管内存（请参阅 [可选：编译 GPU 代码](#Optional:-Compiling-GPU-Code) 了解如何检查您的 GPU 是否支持它），您需要做的就是将托管选项添加到我们的 `-ta` 标志。

`-ta=tesla:managed`

### 分析托管内存代码

In [ ]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_managed ./laplace_managed

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并 <mark>右键单击</mark> [此处](laplace_managed.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看分析器报告，请参阅 [如何查看报告](../../../module2/assets/Fortran/README.ipynb) 部分。

![managed1.PNG](../images/managed1.png)

此时，请随意浏览分析器。然后，当您准备好时，让我们放大。

![managed2.PNG](../images/managed2.png)

我们可以看到，我们的计算区域（我们的 `calcNext` 和 `swap` 函数调用）现在更加紧密。它们之间的数据传输明显减少。通过使用托管内存，编译器可以避免在 CPU 和 GPU 之间来回传输数据。在下一个模块中，我们将学习如何手动执行此操作（这将稍微提高性能），但目前，使用托管内存就足够了。

---

## 总结

我们已经学会了如何使用托管内存在 GPU 上运行我们的代码。我们还尝试了自己管理数据，但效果并不如我们所希望的那样好。在下一个模块中，我们将扩展这些数据概念并学习管理数据的正确方法，并且不再需要依赖编译器。

---

## 额外任务

1. 如果您想要一些关于使用 OpenACC 并行化代码的额外课程，OpenACC YouTube 页面上有一个 OpenACC 简介视频系列。该系列的第三和第四个视频涵盖了本实验中涵盖的很多内容。

[OpenACC 并行编程简介 - 第 3 部分](https://youtu.be/Pcc3O6h-YPE)

[OpenACC 并行编程简介 - 第 4 部分](https://youtu.be/atXtVCHq8iw)

---

## 实验后总结

如果您想下载此实验以供以后查看，建议您转到浏览器的文件菜单（而不是 Jupyter 笔记本文件菜单）并保存完整的网页。这将确保图像也被复制下来。

您还可以执行以下单元块来创建您正在处理的文件的 zip 文件，并使用下面的链接下载它。

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**执行上述 zip 命令后**，您应该能够通过按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](openacc_files.zip) 并选择*将链接另存为*来下载并保存 zip 文件。

# 许可

本材料由 NVIDIA Corporation 根据知识共享署名 4.0 国际 (CC BY 4.0) 发布。